In [3]:
import pandas as pd

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

""" 
Fonte dos dados:
https://www.portaltransparencia.gov.br/download-de-dados/orcamento-despesa/

Extraído direto por ano:
https://www.portaltransparencia.gov.br/download-de-dados/orcamento-despesa/{ano}
"""

dataset_path = '../datasets/raw/'
dataset_base_name = 'OrcamentoDespesa'
available_years = [
    '2014',
    '2015',
    '2016',
    '2017',
    '2018',
    '2019',
    '2020',
    '2021',
    # '2022',  # Disponivel mas incompleto
]
columns = ['EXERCÍCIO', 'CÓDIGO ÓRGÃO SUPERIOR', 'NOME ÓRGÃO SUPERIOR',
       'CÓDIGO ÓRGÃO SUBORDINADO', 'NOME ÓRGÃO SUBORDINADO',
       'CÓDIGO UNIDADE ORÇAMENTÁRIA', 'NOME UNIDADE ORÇAMENTÁRIA',
       'CÓDIGO FUNÇÃO', 'NOME FUNÇÃO', 'CÓDIGO SUBFUNÇÃO', 'NOME SUBFUNÇÃO',
       'CÓDIGO PROGRAMA ORÇAMENTÁRIO', 'NOME PROGRAMA ORÇAMENTÁRIO',
       'CÓDIGO AÇÃO', 'NOME AÇÃO', 'CÓDIGO CATEGORIA ECONÔMICA',
       'NOME CATEGORIA ECONÔMICA', 'CÓDIGO GRUPO DE DESPESA',
       'NOME GRUPO DE DESPESA', 'CÓDIGO ELEMENTO DE DESPESA',
       'NOME ELEMENTO DE DESPESA', 'ORÇAMENTO INICIAL (R$)',
       'ORÇAMENTO ATUALIZADO (R$)', 'ORÇAMENTO EMPENHADO (R$)',
       'ORÇAMENTO REALIZADO (R$)']

"""
Em 2020 houve uma despesa com CATEGORIA ECONÔMICA e GRUPO DE DESPESA inválidos.
Isto faz com que o pandas assuma que a coluna é do tipo `object`, gerando erros na junção dos datasets, 
portanto as colunas 'CÓDIGO CATEGORIA ECONÔMICA' e 'CÓDIGO GRUPO DE DESPESA' serão convertidas para `object`
"""
dtype_conversions = {
    'CÓDIGO CATEGORIA ECONÔMICA': 'object',
    'CÓDIGO GRUPO DE DESPESA': 'object',
}

raw_df = pd.DataFrame(columns=columns)
df_shapes = []

for year in available_years:
    ydf = pd.read_csv(f"{dataset_path}{year}_{dataset_base_name}.csv", sep=';', encoding='Latin-1')
    ydf = ydf.astype(dtype_conversions)

    raw_df = pd.merge(raw_df, ydf, how='outer')
    df_shapes.append(ydf.shape)

/tmp/ipykernel_13589/4015443860.py:56: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  raw_df = pd.merge(raw_df, ydf, how='outer')
/tmp/ipykernel_13589/4015443860.py:56: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  raw_df = pd.merge(raw_df, ydf, how='outer')
/tmp/ipykernel_13589/4015443860.py:56: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  raw_df = pd.merge(raw_df, ydf, how='outer')
/tmp/ipykernel_13589/4015443860.py:56: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  raw_df = pd.merge(raw_df, ydf, how='outer')
/tmp/ipykernel_13589/4015443860.py:56: FutureWarning: In a f

In [4]:
# Checagem simples da quantidade de linhas, para ter mais confiabilidade no dataset final
total_line_count = 0
for shape in df_shapes:
    total_line_count += shape[0]

print(total_line_count, raw_df.shape)
print(total_line_count == raw_df.shape[0])

# Mantém cópia do dataset original por segurança
df = raw_df

214908 (214908, 25)
True


In [6]:
# Remove colunas de CODIGO, pois são redundantes
code_columns = [col for col in df.columns if 'CÓDIGO' in col]
df = df.drop(code_columns, axis=1)

In [7]:
# Corrige separador decimal
df['ORÇAMENTO INICIAL (R$)'] = df['ORÇAMENTO INICIAL (R$)'].str.replace(',', '.')
df['ORÇAMENTO ATUALIZADO (R$)'] = df['ORÇAMENTO ATUALIZADO (R$)'].str.replace(',', '.')
df['ORÇAMENTO EMPENHADO (R$)'] = df['ORÇAMENTO EMPENHADO (R$)'].str.replace(',', '.')
df['ORÇAMENTO REALIZADO (R$)'] = df['ORÇAMENTO REALIZADO (R$)'].str.replace(',', '.')

# Converte colunas de ORÇAMENTO em float
budget_columns = [col for col in df.columns if 'ORÇAMENTO ' in col]
budget_dtypes_dict = { col:'float' for col in budget_columns }
df = df.astype(budget_dtypes_dict)

In [8]:
# Renomeia colunas
column_names = {
    'EXERCÍCIO': 'exercicio', 
    'NOME ÓRGÃO SUPERIOR': 'nome_orgao_superior', 
    'NOME ÓRGÃO SUBORDINADO': 'nome_orgao_subordinado', 
    'NOME UNIDADE ORÇAMENTÁRIA': 'nome_unidade_orcamentaria', 
    'NOME FUNÇÃO': 'nome_funcao', 
    'NOME SUBFUNÇÃO': 'nome_subfuncao', 
    'NOME PROGRAMA ORÇAMENTÁRIO': 'nome_programa_orcamentario', 
    'NOME AÇÃO': 'nome_acao', 
    'NOME CATEGORIA ECONÔMICA': 'nome_categoria_economica', 
    'NOME GRUPO DE DESPESA': 'nome_grupo_despesa', 
    'NOME ELEMENTO DE DESPESA': 'nome_elemento_despesa', 
    'ORÇAMENTO INICIAL (R$)': 'orcamento_inicial',
    'ORÇAMENTO ATUALIZADO (R$)': 'orcamento_atualizado',
    'ORÇAMENTO EMPENHADO (R$)': 'orcamento_empenhado',
    'ORÇAMENTO REALIZADO (R$)': 'orcamento_realizado',
}

df.rename(columns=column_names, inplace=True)

In [9]:
# Visualização de informações básicas do dataset
print(df.info())
df.sample(5)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 214908 entries, 0 to 214907
Data columns (total 15 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   exercicio                   214908 non-null  int64  
 1   nome_orgao_superior         214908 non-null  object 
 2   nome_orgao_subordinado      214908 non-null  object 
 3   nome_unidade_orcamentaria   214908 non-null  object 
 4   nome_funcao                 214908 non-null  object 
 5   nome_subfuncao              214908 non-null  object 
 6   nome_programa_orcamentario  214908 non-null  object 
 7   nome_acao                   214908 non-null  object 
 8   nome_categoria_economica    214908 non-null  object 
 9   nome_grupo_despesa          214908 non-null  object 
 10  nome_elemento_despesa       214908 non-null  object 
 11  orcamento_inicial           214908 non-null  float64
 12  orcamento_atualizado        214908 non-null  float64
 13  orcamento_empe

,exercicio,nome_orgao_superior,nome_orgao_subordinado,nome_unidade_orcamentaria,nome_funcao,nome_subfuncao,nome_programa_orcamentario,nome_acao,nome_categoria_economica,nome_grupo_despesa,nome_elemento_despesa,orcamento_inicial,orcamento_atualizado,orcamento_empenhado,orcamento_realizado
131987,2018,Ministério da Defesa,Ministério da Defesa - Unidades com vínculo di...,FUNDO DE ADM. DO HOSPITAL DAS FORCAS ARMADAS,Defesa nacional,Assistência hospitalar e ambulatorial,PROGRAMA DE GESTAO E MANUTENCAO DO MINISTERIO ...,PRESTACAO DE SERVICOS MEDICO-HOSPITALARES E DE...,DESPESAS CORRENTES,Outras Despesas Correntes,Indenizações e Restituições,108796.23,108796.23,108796.23,105257.27
168646,2020,Ministério da Educação,Universidade Federal do Pará,REC. SOB SUP. HOSP. UNIV. BETTINA FERRO SOUZA,Previdência social,Previdência do regime estatutário,PROGRAMA DE GESTAO E MANUTENCAO DO PODER EXECU...,APOSENTADORIAS E PENSOES CIVIS DA UNIAO,DESPESAS CORRENTES,Pessoal e Encargos Sociais,Não informado,616514.00,0.00,0.00,0.00
36564,2015,Ministério da Educação,Universidade Federal de Lavras,UNIVERSIDADE FEDERAL DE LAVRAS,Educação,Ensino superior,"EDUCACAO SUPERIOR - GRADUACAO, POS-GRADUACAO, ...","FOMENTO AS ACOES DE GRADUACAO, POS-GRADUACAO, ...",DESPESAS DE CAPITAL,Investimentos,Não informado,173107.00,173107.00,0.00,0.00
92570,2017,Ministério da Educação,Fundação Universidade Federal de Pelotas,FUNDACAO UNIVERSIDADE FEDERAL DE PELOTAS,Educação,Ensino superior,EDUCACAO DE QUALIDADE PARA TODOS,FUNCIONAMENTO DE INSTITUICOES FEDERAIS DE ENSI...,DESPESAS CORRENTES,Outras Despesas Correntes,Diárias - Civil,509361.64,509361.64,509184.64,509184.64
157242,2019,Ministério da Defesa,Fundo Aeronáutico,FUNDO AERONAUTICO,Defesa nacional,Defesa aérea,DEFESA NACIONAL,APRESTAMENTO DA AERONAUTICA,DESPESAS DE CAPITAL,Inversões Financeiras,Aquisição de bens para revenda,4229991.04,4229991.04,0.00,0.00


In [12]:
# Exporta o arquivo pré-processado, com dados agregados de todos os anos
from datetime import datetime

output_path = '../datasets/preprocessed/'
t = datetime.now()

df.to_csv(f"{output_path}{dataset_base_name}_{t}.csv", sep=';', encoding='utf-8', index = False)